In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [5]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
vgg19 = \
    model_development.vgg19(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = \
    model_development.train_classification_model(training_phase = 1, model = vgg19, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.9445 - accuracy: 0.5224 - precision: 0.4818 - recall: 0.4077
Epoch 1: val_accuracy improved from -inf to 0.55522, saving model to ../models/weights\VGG19_1.00.00_fs.hdf5
119/119 [==============================] - 31s 202ms/step - loss: 0.9445 - accuracy: 0.5224 - precision: 0.4818 - recall: 0.4077 - val_loss: 0.6899 - val_accuracy: 0.5552 - val_precision: 0.4936 - val_recall: 0.7945
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.6557 - accuracy: 0.6259 - precision: 0.6275 - recall: 0.4738
Epoch 2: val_accuracy improved from 0.55522 to 0.57015, saving model to ../models/weights\VGG19_1.00.00_fs.hdf5
119/119 [==============================] - 22s 183ms/step - loss: 0.6557 - accuracy: 0.6259 - precision: 0.6275 - recall: 0.4738 - val_loss: 0.7022 - val_accuracy: 0.5701 - val_precision: 0.5060 - val_recall: 0.5822
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.6421 -

In [6]:
##
# Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/VGG19_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.6094 - accuracy: 0.6649 - precision: 0.6890 - recall: 0.5046
Epoch 1: val_accuracy improved from -inf to 0.62687, saving model to ../models/weights\VGG19_1.00.00_ss.hdf5
119/119 [==============================] - 46s 353ms/step - loss: 0.6094 - accuracy: 0.6649 - precision: 0.6890 - recall: 0.5046 - val_loss: 0.6273 - val_accuracy: 0.6269 - val_precision: 0.5574 - val_recall: 0.6986
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5721 - accuracy: 0.6871 - precision: 0.7478 - recall: 0.4897
Epoch 2: val_accuracy did not improve from 0.62687
119/119 [==============================] - 40s 336ms/step - loss: 0.5721 - accuracy: 0.6871 - precision: 0.7478 - recall: 0.4897 - val_loss: 0.8521 - val_accuracy: 0.4358 - val_precision: 0.4354 - val_recall: 0.9932
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5672 - accuracy: 0.6966 - precision: 0.7409 - recall: 0.5308
Epoch 

In [7]:
##
# Clear memories after model training:
#
model_development.clear_memory()